In [1]:
from __future__ import absolute_import
import os
import sys
import traceback

import numpy
import theano

os.environ['THEANO_FLAGS'] = "device=cuda%s,floatX=float32" % "5"

import theano.tensor as t_func
from theano.tensor.shared_randomstreams import RandomStreams

try:
    import cPickle as pickle
except ImportError:
    import pickle as pickle


from ExpUtils import *
from theano_func.CostFunc import get_cost_type_semi
from theano_func.source import optimizers
from theano_func.source import costs
from theano_func.models.fnn_mnist_semisup import FNN_MNIST
from collections import OrderedDict
from theano_func import load_data
%load_ext autoreload
%autoreload 2

In [5]:
from tensorboardX import SummaryWriter
dir_path = os.path.join(os.environ['HOME'], 'project/runs', 'VAT-theano-semi/mnist/b')
writer = SummaryWriter(log_dir=dir_path)
debug = True

In [26]:
from theano_func.models.fnn import FNN
from theano_func.source import layers as L

def init_linear(shape):
    print(shape[::-1])
    fan_in = shape[0]
    s = 1.0 * np.sqrt(6.0 / fan_in)
    w = np.random.uniform(-s, s, shape[::-1])# .astype("float32")
    w = w.T
    if debug:
        print(shape, w.sum())
    return w

class FNN_MNIST(FNN):
    def __init__(self, layer_sizes):
        self.linear_layers = []
        self.bn_layers = []
        self.act_layers = []
        self.params = []
        layers = zip(layer_sizes[:-1], layer_sizes[1:])
        for i, (m, n) in enumerate(layers):
            l = L.Linear(size=(m, n), initial_W=init_linear((m, n)))
            bn = L.BatchNormalization(size=(n))
            self.linear_layers.append(l)
            self.bn_layers.append(bn)
            self.params += l.params + bn.params
        for i in range(len(self.linear_layers) - 1):
            self.act_layers.append(L.relu)
        self.act_layers.append(L.softmax)

    def forward_for_finetuning_batch_stat(self, input):
        return self.forward(input, finetune=True)

    def forward_no_update_batch_stat(self, input, train=True):
        return self.forward(input, train, False)

    def forward(self, input, train=True, update_batch_stat=True, finetune=False):
        h = input
        for l, bn, act in zip(self.linear_layers, self.bn_layers, self.act_layers):
            h = l(h)
            if bn is not None:
                h = bn(h, train=train, update_batch_stat=update_batch_stat, finetune=finetune)
            h = act(h)
        return h

In [25]:
np.random.seed(1)
m = FNN_MNIST([784, 1200, 1200, 10])

((784, 1200), -13.070688810470633)
((1200, 1200), -44.62377855519668)
((1200, 10), -3.5299364412835765)


In [27]:
def train(args):

    numpy.random.seed(int(args['seed']))

    dataset = load_data.load_mnist_for_semi_sup(n_l=int(args['size']),
                                                n_v=int(args['num_validation_samples']))

    x_train, t_train, ul_x_train = dataset[0]
    
    x_test, t_test = dataset[2]

    numpy.random.seed(int(args['seed']))
    layer_sizes = [int(layer_size) for layer_size in args['layer_sizes'].split('-')]
    model = FNN_MNIST(layer_sizes=layer_sizes)

    x = t_func.matrix()
    ul_x = t_func.matrix()
    t = t_func.ivector()

    cost_semi, sup, unsup, err = get_cost_type_semi(model, x, t, ul_x, args)
    nll = costs.cross_entropy_loss(x=x, t=t, forward_func=model.forward_test)
    error = costs.error(x=x, t=t, forward_func=model.forward_test)

    optimizer = optimizers.ADAM(cost=cost_semi, params=model.params, alpha=float(args['lr']))

    index = t_func.iscalar()
    ul_index = t_func.iscalar()
    batch_size = int(args['batch_size'])
    ul_batch_size = int(args['ul_batch_size'])

    f_train = theano.function(inputs=[index, ul_index], 
                              outputs=[cost_semi, sup, unsup, err, x, ul_x], updates=optimizer.updates,
                              givens={
                                  x: x_train[batch_size * index:batch_size * (index + 1)],
                                  t: t_train[batch_size * index:batch_size * (index + 1)],
                                  ul_x: ul_x_train[ul_batch_size * ul_index:ul_batch_size * (ul_index + 1)]},
                              on_unused_input='ignore')
    f_nll_train = theano.function(inputs=[index], outputs=nll,
                                  givens={
                                      x: x_train[batch_size * index:batch_size * (index + 1)],
                                      t: t_train[batch_size * index:batch_size * (index + 1)]})
    f_nll_test = theano.function(inputs=[index], outputs=nll,
                                 givens={
                                     x: x_test[batch_size * index:batch_size * (index + 1)],
                                     t: t_test[batch_size * index:batch_size * (index + 1)]})

    f_error_train = theano.function(inputs=[index], outputs=error,
                                    givens={
                                        x: x_train[batch_size * index:batch_size * (index + 1)],
                                        t: t_train[batch_size * index:batch_size * (index + 1)]})
    f_error_test = theano.function(inputs=[index], outputs=error,
                                   givens={
                                       x: x_test[batch_size * index:batch_size * (index + 1)],
                                       t: t_test[batch_size * index:batch_size * (index + 1)]})

    f_lr_decay = theano.function(inputs=[], outputs=optimizer.alpha,
                                 updates={optimizer.alpha: theano.shared(
                                     numpy.array(args['lr_decay']).astype(
                                         theano.config.floatX)) * optimizer.alpha})

    # Shuffle training set
    randix = RandomStreams(seed=numpy.random.randint(1234)).permutation(n=x_train.shape[0])
    update_permutation = OrderedDict()
    update_permutation[x_train] = x_train[randix]
    update_permutation[t_train] = t_train[randix]
    f_permute_train_set = theano.function(inputs=[], outputs=x_train, updates=update_permutation)

    # Shuffle unlabeled training set
    ul_randix = RandomStreams(seed=numpy.random.randint(1234)).permutation(n=ul_x_train.shape[0])
    update_ul_permutation = OrderedDict()
    update_ul_permutation[ul_x_train] = ul_x_train[ul_randix]
    f_permute_ul_train_set = theano.function(inputs=[], outputs=ul_x_train, updates=update_ul_permutation)

    statuses = {'nll_train': [], 'error_train': [], 'nll_test': [], 'error_test': []}

    n_train = x_train.get_value().shape[0]
    n_test = x_test.get_value().shape[0]
    n_ul_train = ul_x_train.get_value().shape[0]

    l_i = 0
    ul_i = 0
    for epoch in range(int(args['epochs'])):

        f_permute_train_set()
        f_permute_ul_train_set()
        for it in range(int(args['iterations'])):
            loss, sup, unsup, pert_err, x, ul_x = f_train(l_i, ul_i)
            os.path.join(os.environ['HOME'], 'project/runs')
            l_i = 0 if l_i >= n_train / batch_size - 1 else l_i + 1
            ul_i = 0 if ul_i >= n_ul_train / ul_batch_size - 1 else ul_i + 1
            if it % 100 == 0:
                print("loss", loss)
                print("sup", sup)
                print("unsup", unsup)
                print("pert err", pert_err)
                print("x", x.sum())
                print(ul_x.shape)
                print("unl x", ul_x.sum())
                writer.add_scalar("Train/TotalLoss", loss, epoch * int(args['iterations']) + it)

        sum_nll_train = numpy.sum(numpy.array([f_nll_train(i) for i in range(n_train // batch_size)])) * batch_size
        sum_error_train = numpy.sum(numpy.array([f_error_train(i) for i in range(n_train // batch_size)]))
        sum_nll_test = numpy.sum(numpy.array([f_nll_test(i) for i in range(n_test // batch_size)])) * batch_size
        sum_error_test = numpy.sum(numpy.array([f_error_test(i) for i in range(n_test // batch_size)]))
        statuses['nll_train'].append(sum_nll_train / n_train)
        statuses['error_train'].append(sum_error_train)
        statuses['nll_test'].append(sum_nll_test / n_test)
        statuses['error_test'].append(sum_error_test)
        wlog("[Epoch] %d" % epoch)
        acc = 1 - 1.0*statuses['error_test'][-1]/n_test
        wlog("nll_train: %f error_train : %d nll_test : %f error_test : %d accuracy:%f" % (
            statuses['nll_train'][-1], statuses['error_train'][-1], statuses['nll_test'][-1],
            statuses['error_test'][-1], acc))
        if args["vis"]:
            writer.add_scalar("Train/Loss", statuses['nll_train'][-1], epoch * int(args['iterations']))
            writer.add_scalar("Train/Error", statuses['error_train'][-1], epoch * int(args['iterations']))
            writer.add_scalar("Test/Loss", statuses['nll_test'][-1], epoch * int(args['iterations']))
            writer.add_scalar("Test/Acc", acc, epoch * int(args['iterations']))
        f_lr_decay()
    # fine_tune batch stat
    f_fine_tune = theano.function(inputs=[ul_index], outputs=model.forward_for_finetuning_batch_stat(x),
                                  givens={x: ul_x_train[ul_batch_size * ul_index:ul_batch_size * (ul_index + 1)]})
    [f_fine_tune(i) for i in range(n_ul_train // ul_batch_size)]

    sum_nll_train = numpy.sum(numpy.array([f_nll_train(i) for i in range(n_train // batch_size)])) * batch_size
    sum_error_train = numpy.sum(numpy.array([f_error_train(i) for i in range(n_train // batch_size)]))
    sum_nll_test = numpy.sum(numpy.array([f_nll_test(i) for i in range(n_test // batch_size)])) * batch_size
    sum_error_test = numpy.sum(numpy.array([f_error_test(i) for i in range(n_test // batch_size)]))
    statuses['nll_train'].append(sum_nll_train / n_train)
    statuses['error_train'].append(sum_error_train)
    statuses['nll_test'].append(sum_nll_test / n_test)
    statuses['error_test'].append(sum_error_test)
    wlog("final nll_train: %f error_train: %d nll_test: %f error_test: %d accuracy:%f" % (
        statuses['nll_train'][-1], statuses['error_train'][-1], statuses['nll_test'][-1],
        statuses['error_test'][-1], 1 - 1.0*statuses['error_test'][-1]/n_test))

    error_test = numpy.array(statuses['error_test'])


In [28]:

arg = {'seed': 1,
       'size': 100,
       'vis': False,
       'epochs': 10,
       'iterations': 1,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'VAT_f',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.002
      }
train(arg)

(1200, 784)
((784, 1200), -13.070688810470633)
(1200, 1200)
((1200, 1200), -44.62377855519668)
(10, 1200)
((1200, 10), -3.5299364412835765)
('loss', array(4.6505536))
('sup', array(2.52342667))
('unsup', array(2.12712693))
('pert err', array(138))
('x', 10491.24609375)
(250, 784)
('unl x', 25638.88671875)


KeyboardInterrupt: 

In [5]:

arg = {'seed': 1,
       'size': 100,
       'vis': False,
       'epochs': 100,
       'iterations': 1,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)

2.880021582795081
(250, 784)
25141.703125


2019-04-06 01:50:09,985 - <ipython-input-4-4b8d37f5ef7f>[line:101]: [Epoch] 0
2019-04-06 01:50:09,986 - <ipython-input-4-4b8d37f5ef7f>[line:105]: nll_train: 1.788823 error_train : 25 nll_test : 1.971886 error_test : 5213 accuracy:0.478700


0.6267069340277571
(250, 784)
25494.30859375


2019-04-06 01:50:17,351 - <ipython-input-4-4b8d37f5ef7f>[line:101]: [Epoch] 1
2019-04-06 01:50:17,352 - <ipython-input-4-4b8d37f5ef7f>[line:105]: nll_train: 1.501638 error_train : 9 nll_test : 1.807326 error_test : 3998 accuracy:0.600200


0.4610604825121197
(250, 784)
24649.28515625


...

2019-04-06 01:58:36,003 - <ipython-input-4-4b8d37f5ef7f>[line:101]: [Epoch] 97
2019-04-06 01:58:36,004 - <ipython-input-4-4b8d37f5ef7f>[line:105]: nll_train: 0.281065 error_train : 0 nll_test : 1.166778 error_test : 2456 accuracy:0.754400


0.26877786623094024
(250, 784)
25875.08203125


2019-04-06 01:58:41,701 - <ipython-input-4-4b8d37f5ef7f>[line:101]: [Epoch] 98
2019-04-06 01:58:41,702 - <ipython-input-4-4b8d37f5ef7f>[line:105]: nll_train: 0.281037 error_train : 0 nll_test : 1.166760 error_test : 2456 accuracy:0.754400


0.26876958934669476
(250, 784)
24702.5703125


2019-04-06 01:58:47,286 - <ipython-input-4-4b8d37f5ef7f>[line:101]: [Epoch] 99
2019-04-06 01:58:47,287 - <ipython-input-4-4b8d37f5ef7f>[line:105]: nll_train: 0.281011 error_train : 0 nll_test : 1.166742 error_test : 2456 accuracy:0.754400
2019-04-06 01:59:21,950 - <ipython-input-4-4b8d37f5ef7f>[line:127]: final nll_train: 0.069420 error_train: 0 nll_test: 0.929169 error_test: 2434 accuracy:0.756600


In [7]:

arg = {'seed': 1,
       'size': 100,
       'vis': False,
       'epochs': 100,
       'iterations': 1,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)

2019-04-01 19:41:34,539 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 0
2019-04-01 19:41:34,540 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.788823 error_train : 25 nll_test : 1.971886 error_test : 5213 accuracy:0.478700
2019-04-01 19:41:38,691 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 1
2019-04-01 19:41:38,693 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.501638 error_train : 9 nll_test : 1.807326 error_test : 3998 accuracy:0.600200
2019-04-01 19:41:42,945 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 2
2019-04-01 19:41:42,948 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.363042 error_train : 4 nll_test : 1.735865 error_test : 3478 accuracy:0.652200
2019-04-01 19:41:46,914 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 3
2019-04-01 19:41:46,915 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.287941 error_train : 1 nll_test : 1.703039 error_test : 3177 accuracy:0.682300
2019-04-01 19:41:51,025 - <ipython-input-4-df82ab41

2019-04-01 19:44:09,787 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 35
2019-04-01 19:44:09,788 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.373939 error_train : 0 nll_test : 1.237688 error_test : 2472 accuracy:0.752800
2019-04-01 19:44:14,068 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 36
2019-04-01 19:44:14,070 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.365216 error_train : 0 nll_test : 1.231295 error_test : 2467 accuracy:0.753300
2019-04-01 19:44:20,021 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 37
2019-04-01 19:44:20,023 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.357283 error_train : 0 nll_test : 1.225427 error_test : 2467 accuracy:0.753300
2019-04-01 19:44:26,185 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 38
2019-04-01 19:44:26,194 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.350080 error_train : 0 nll_test : 1.220049 error_test : 2467 accuracy:0.753300
2019-04-01 19:44:33,824 - <ipython-input-4-df82a

2019-04-01 19:47:05,025 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 70
2019-04-01 19:47:05,026 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.283971 error_train : 0 nll_test : 1.168929 error_test : 2454 accuracy:0.754600
2019-04-01 19:47:09,016 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 71
2019-04-01 19:47:09,017 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.283697 error_train : 0 nll_test : 1.168720 error_test : 2455 accuracy:0.754500
2019-04-01 19:47:13,196 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 72
2019-04-01 19:47:13,197 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.283448 error_train : 0 nll_test : 1.168530 error_test : 2455 accuracy:0.754500
2019-04-01 19:47:17,113 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 73
2019-04-01 19:47:17,115 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 0.283221 error_train : 0 nll_test : 1.168358 error_test : 2454 accuracy:0.754600
2019-04-01 19:47:21,271 - <ipython-input-4-df82a

In [6]:

arg = {'seed': 1,
       'size': 100,
       'vis': False,
       'epochs': 20,
       'iterations': 1,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)

2019-04-01 18:59:40,970 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 0
2019-04-01 18:59:40,971 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.788823 error_train : 25 nll_test : 1.971886 error_test : 5213 accuracy:0.478700
2019-04-01 18:59:44,884 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 1
2019-04-01 18:59:44,886 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.501638 error_train : 9 nll_test : 1.807326 error_test : 3998 accuracy:0.600200
2019-04-01 18:59:48,923 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 2
2019-04-01 18:59:48,927 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.363042 error_train : 4 nll_test : 1.735865 error_test : 3478 accuracy:0.652200
2019-04-01 18:59:52,821 - <ipython-input-4-df82ab415009>[line:97]: [Epoch] 3
2019-04-01 18:59:52,822 - <ipython-input-4-df82ab415009>[line:101]: nll_train: 1.287941 error_train : 1 nll_test : 1.703039 error_test : 3177 accuracy:0.682300
2019-04-01 18:59:56,715 - <ipython-input-4-df82ab41

In [7]:

arg = {'seed': 1,
       'size': 1000,
       'vis': False,
       'epochs': 100,
       'iterations': 400,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)

2019-03-11 01:40:13,528 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 0
2019-03-11 01:40:13,531 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.127680 error_train : 0 nll_test : 0.473476 error_test : 704 accuracy:0.929600
2019-03-11 01:41:03,025 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 1
2019-03-11 01:41:03,028 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.063812 error_train : 0 nll_test : 0.392298 error_test : 725 accuracy:0.927500
2019-03-11 01:41:51,856 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 2
2019-03-11 01:41:51,858 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.040191 error_train : 0 nll_test : 0.344783 error_test : 697 accuracy:0.930300
2019-03-11 01:42:37,339 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 3
2019-03-11 01:42:37,341 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.026402 error_train : 0 nll_test : 0.318617 error_test : 697 accuracy:0.930300
2019-03-11 01:43:23,362 - <ipython-input-6-aee519e8df5c>

2019-03-11 02:07:33,178 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 35
2019-03-11 02:07:33,180 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000158 error_train : 0 nll_test : 0.243054 error_test : 722 accuracy:0.927800
2019-03-11 02:08:19,538 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 36
2019-03-11 02:08:19,540 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000145 error_train : 0 nll_test : 0.245175 error_test : 732 accuracy:0.926800
2019-03-11 02:09:05,825 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 37
2019-03-11 02:09:05,827 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000137 error_train : 0 nll_test : 0.244926 error_test : 728 accuracy:0.927200
2019-03-11 02:09:52,091 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 38
2019-03-11 02:09:52,093 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000116 error_train : 0 nll_test : 0.247328 error_test : 719 accuracy:0.928100
2019-03-11 02:10:36,051 - <ipython-input-6-aee519e8d

2019-03-11 02:34:44,930 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 70
2019-03-11 02:34:44,931 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000020 error_train : 0 nll_test : 0.263617 error_test : 735 accuracy:0.926500
2019-03-11 02:35:30,961 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 71
2019-03-11 02:35:30,963 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000020 error_train : 0 nll_test : 0.266969 error_test : 730 accuracy:0.927000
2019-03-11 02:36:17,490 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 72
2019-03-11 02:36:17,492 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000019 error_train : 0 nll_test : 0.265538 error_test : 725 accuracy:0.927500
2019-03-11 02:37:03,996 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 73
2019-03-11 02:37:03,998 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000018 error_train : 0 nll_test : 0.266069 error_test : 738 accuracy:0.926200
2019-03-11 02:37:46,480 - <ipython-input-6-aee519e8d

In [8]:

arg = {'seed': 1,
       'size': 10000,
       'vis': False,
       'epochs': 100,
       'iterations': 400,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)

2019-03-11 03:50:32,033 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 0
2019-03-11 03:50:32,035 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.154206 error_train : 0 nll_test : 0.265320 error_test : 283 accuracy:0.971700
2019-03-11 03:51:51,301 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 1
2019-03-11 03:51:51,303 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.070784 error_train : 0 nll_test : 0.179951 error_test : 257 accuracy:0.974300
2019-03-11 03:53:06,957 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 2
2019-03-11 03:53:06,959 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.039508 error_train : 0 nll_test : 0.144496 error_test : 245 accuracy:0.975500
2019-03-11 03:54:25,682 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 3
2019-03-11 03:54:25,683 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.027010 error_train : 0 nll_test : 0.124050 error_test : 246 accuracy:0.975400
2019-03-11 03:55:33,338 - <ipython-input-6-aee519e8df5c>

2019-03-11 04:23:39,605 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 35
2019-03-11 04:23:39,607 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000174 error_train : 0 nll_test : 0.082221 error_test : 222 accuracy:0.977800
2019-03-11 04:24:29,983 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 36
2019-03-11 04:24:29,985 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000154 error_train : 0 nll_test : 0.084303 error_test : 234 accuracy:0.976600
2019-03-11 04:25:19,331 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 37
2019-03-11 04:25:19,332 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000152 error_train : 0 nll_test : 0.084445 error_test : 230 accuracy:0.977000
2019-03-11 04:26:10,681 - <ipython-input-6-aee519e8df5c>[line:97]: [Epoch] 38
2019-03-11 04:26:10,683 - <ipython-input-6-aee519e8df5c>[line:101]: nll_train: 0.000130 error_train : 0 nll_test : 0.084206 error_test : 234 accuracy:0.976600
2019-03-11 04:27:01,937 - <ipython-input-6-aee519e8d

KeyboardInterrupt: 

In [ ]:

arg = {'seed': 1,
       'size': 50000,
       'vis': False,
       'epochs': 100,
       'iterations': 400,
       'lr_decay': 0.95,
       'batch_size': 100,
       'ul_batch_size': 250,
       'num_validation_samples': 1000,
       'layer_sizes': '784-1200-1200-10',
       'dataset': 'mnist',
       'epsilon': 0.3,
       'cost_type': 'MLE',
       'norm_constraint': 'L2',
       'num_power_iter': 1,
       'lamb': 1,
       'lr': 0.001
      }
train(arg)